In [1]:
#custom stuff
print("ok")

ok


In [2]:
import numpy as np
import tensorflow as tf 

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,Subtract,Reshape
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Conv2D,MaxPooling2D,Input,Lambda,GlobalMaxPooling2D
from keras.regularizers import l2
from keras import backend as K
from keras.applications.vgg16 import VGG16
from skimage.io import imsave

from matplotlib.pyplot import imread
from skimage.transform import rescale, resize
import os
from keras.models import load_model
from sklearn.model_selection import train_test_split

print("loaded libs.")

loaded libs.


In [3]:
# Dataset parameters 
dataset_path = '../../datasets/VisualPhish/'
reshape_size = [224,224,3]
phishing_test_size = 0.4
num_targets = 25

# Model parameters
input_shape = [224,224,3]
margin = 2.2
new_conv_params = [3,3,512]

# Training parameters
start_lr = 0.00002
output_dir = './'
saved_model_name = 'model'
save_interval = 2000
batch_size = 32 
n_iter = 1000 #50000
lr_interval = 300

#!find datasets/trusted_list/ -name *.png | wc -l
#!find datasets/phishing/ -name *.png | wc -l
#2031
#3821

# Load dataset:
    - Load training screenshots per website
    - Load Phishing screenshots per website 

In [4]:

def read_imgs_per_website(data_path,targets,imgs_num,reshape_size,start_target_count):
    all_imgs = np.zeros(shape=[imgs_num,224,224,3])
    all_labels = np.zeros(shape=[imgs_num,1])
    
    all_file_names = []
    targets_list = targets.splitlines()
    count = 0
    for i in range(0,len(targets_list)):
        target_path = data_path + targets_list[i]
        print("[{}]loading:{}".format(i,target_path))
        file_names = sorted(os.listdir(target_path))
        for j in range(0,len(file_names)):
            try:
                img = imread(target_path+'/'+file_names[j])
                img = img[:,:,0:3]
                all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                all_labels[count,:] = i + start_target_count
                all_file_names.append(file_names[j])
                count = count + 1
            except:
                #some images were saved with a wrong extensions 
                try:
                    img = imread(target_path+'/'+file_names[j],format='jpeg')
                    img = img[:,:,0:3]
                    all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                    all_labels[count,:] = i + start_target_count
                    all_file_names.append(file_names[j])
                    count = count + 1
                except:
                    print('failed at:')
                    print('***')
                    print(file_names[j])
                    break 
    return all_imgs,all_labels,all_file_names



In [5]:
# Read images legit (train)
data_path = dataset_path + 'trusted_list/'
targets_file = open(data_path+'targets.txt', "r")
targets = targets_file.read()
imgs_num = 2031     #9363
all_imgs_train,all_labels_train,all_file_names_train = read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

# Read images phishing
data_path = dataset_path + 'phishing/'
targets_file = open(data_path+'targets.txt', "r")
targets = targets_file.read()
imgs_num = 3821     #1195
all_imgs_test,all_labels_test,all_file_names_test = read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

X_train_legit = all_imgs_train
y_train_legit = all_labels_train

# Split phishing to training and test 
idx = np.arange(all_imgs_test.shape[0])
X_test_phish, X_train_phish, y_test_phish, y_train_phish,idx_test,idx_train = train_test_split(all_imgs_test, all_labels_test,idx, test_size=phishing_test_size)
np.save(output_dir+'test_idx',idx_test)
np.save(output_dir+'train_idx',idx_train)
print("load complete.")

[0]loading:../../datasets/VisualPhish/trusted_list/adobe.com
[1]loading:../../datasets/VisualPhish/trusted_list/github.com
[2]loading:../../datasets/VisualPhish/trusted_list/player.vimeo.com
[3]loading:../../datasets/VisualPhish/trusted_list/wordpress.com
[4]loading:../../datasets/VisualPhish/trusted_list/amazon.com
[5]loading:../../datasets/VisualPhish/trusted_list/google.com
[6]loading:../../datasets/VisualPhish/trusted_list/play.google.com
[7]loading:../../datasets/VisualPhish/trusted_list/wordpress.org
[8]loading:../../datasets/VisualPhish/trusted_list/apache.org
[9]loading:../../datasets/VisualPhish/trusted_list/gravatar.com
[10]loading:../../datasets/VisualPhish/trusted_list/qq.com
[11]loading:../../datasets/VisualPhish/trusted_list/yahoo.com
[12]loading:../../datasets/VisualPhish/trusted_list/apple.com
[13]loading:../../datasets/VisualPhish/trusted_list/itunes.apple.com
[14]loading:../../datasets/VisualPhish/trusted_list/support.google.com
[15]loading:../../datasets/VisualPhish/

# Create Model

In [6]:
def define_triplet_network(input_shape, new_conv_params):
    
    # Input_shape: shape of input images
    # new_conv_params: dimension of the new convolution layer [spatial1,spatial2,channels] 
    
    # Define the tensors for the three input images
    anchor_input = Input(input_shape)
    positive_input = Input(input_shape)
    negative_input = Input(input_shape)
    
    # Use VGG as a base model 
    base_model = VGG16(weights='imagenet',  input_shape=input_shape, include_top=False)

    x = base_model.output
    x = Conv2D(new_conv_params[2],(new_conv_params[0],new_conv_params[1]),activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(2e-4)) (x)
    x = GlobalAveragePooling2D() (x)
    model = Model(inputs=base_model.input, outputs=x)

    # Generate the encodings (feature vectors) for the two images
    encoded_a = model(anchor_input)
    encoded_p = model(positive_input)
    encoded_n = model(negative_input)
    
    mean_layer = Lambda(lambda x: K.mean(x,axis=1))
    
    square_diff_layer = Lambda(lambda tensors:K.square(tensors[0] - tensors[1]))
    square_diff_pos = square_diff_layer([encoded_a,encoded_p])
    square_diff_neg = square_diff_layer([encoded_a,encoded_n])
    
    square_diff_pos_l2 = mean_layer(square_diff_pos)
    square_diff_neg_l2 = mean_layer(square_diff_neg)
    
    # Add a diff layer
    diff = Subtract()([square_diff_pos_l2, square_diff_neg_l2])
    diff = Reshape((1,)) (diff)

    # Connect the inputs with the outputs
    triplet_net = Model(inputs=[anchor_input,positive_input,negative_input],outputs=diff)
    
    # return the model
    return triplet_net

In [7]:
# Define the triplet loss
# Create and compile model 

def custom_loss(margin):
    def loss(y_true,y_pred):
        loss_value = K.maximum(y_true, margin + y_pred)
        loss_value = K.mean(loss_value,axis=0)
        return loss_value
    return loss
def loss(y_true,y_pred):
    loss_value = K.maximum(y_true, margin + y_pred)
    loss_value = K.mean(loss_value,axis=0)
    return loss_value


model = define_triplet_network(input_shape, new_conv_params)
model.summary()

from keras import optimizers
optimizer = optimizers.Adam(lr = start_lr)
model.compile(loss=custom_loss(margin),optimizer=optimizer)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 512)          17074496    input_1[0][0]                    
                                                                 input_2[0][0]              

# Triplet Sampling

In [8]:
# Order random phishing arrays per website (from 0 to 155 target)

def order_random_array(orig_arr,y_orig_arr,targets):
    sorted_arr = np.zeros(orig_arr.shape)
    y_sorted_arr = np.zeros(y_orig_arr.shape)
    count = 0
    for i in range(0,targets):
        for j in range(0,orig_arr.shape[0]):
            if y_orig_arr[j] == i:
                sorted_arr[count,:,:,:] = orig_arr[j,:,:,:]
                y_sorted_arr[count,:] = i
                count = count + 1
    return sorted_arr,y_sorted_arr 

X_test_phish,y_test_phish = order_random_array(X_test_phish,y_test_phish,num_targets)
X_train_phish,y_train_phish = order_random_array(X_train_phish,y_train_phish,num_targets)


# Store the start and end of each target in the phishing set (used later in triplet sampling)
# Not all targets might be in the phishing set 
def targets_start_end(num_target,labels):
    prev_target = labels[0]
    start_end_each_target = np.zeros((num_target,2))
    start_end_each_target[0,0] = labels[0]
    if not labels[0] == 0:
        start_end_each_target[0,0] = -1
        start_end_each_target[0,1] = -1
    count_target = 0
    for i in range(1,labels.shape[0]):
        if not labels[i] == prev_target:
            start_end_each_target[int(labels[i-1]),1] = int(i-1)
            start_end_each_target[int(labels[i]),0] = int(i)
            prev_target = labels[i]
    start_end_each_target[int(labels[-1]),1] = int(labels.shape[0]-1)
    
    for i in range(1,num_target):
        if start_end_each_target[i,0] == 0:
            start_end_each_target[i,0] = -1
            start_end_each_target[i,1] = -1
    return start_end_each_target

labels_start_end_train_phish = targets_start_end(num_targets,y_train_phish)
labels_start_end_test_phish = targets_start_end(num_targets,y_test_phish)


# Store the start and end of each target in the training set (used later in triplet sampling)
def all_targets_start_end(num_target,labels):
    prev_target = 0
    start_end_each_target = np.zeros((num_target,2))
    start_end_each_target[0,0] = 0
    count_target = 0
    for i in range(1,labels.shape[0]):
        if not labels[i] == prev_target:
            start_end_each_target[count_target,1] = i-1
            count_target = count_target + 1
            start_end_each_target[count_target,0] = i
            prev_target = prev_target + 1
    start_end_each_target[num_target-1,1] = labels.shape[0]-1
    return start_end_each_target

labels_start_end_train_legit = all_targets_start_end(num_targets,y_train_legit)


In [9]:
#Sample anchor, positive and negative images 
def pick_first_img_idx(labels_start_end,num_targets):
    random_target = -1
    while (random_target == -1):
        random_target = np.random.randint(low = 0,high = num_targets)
        if labels_start_end[random_target,0] == -1:
            random_target = -1
    return random_target

def pick_pos_img_idx(prob_phish,img_label):
    if np.random.uniform() > prob_phish:
        class_idx_start_end = labels_start_end_train_legit[img_label,:]
        same_idx = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
        img = X_train_legit[same_idx,:]
    else:
        if not labels_start_end_train_phish[img_label,0] == -1:
            class_idx_start_end = labels_start_end_train_phish[img_label,:]
            same_idx = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
            img = X_train_phish[same_idx,:]
        else:
            class_idx_start_end = labels_start_end_train_legit[img_label,:]
            same_idx = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
            img = X_train_legit[same_idx,:]
    return img

def pick_neg_img(anchor_idx,num_targets):
    if anchor_idx == 0:
        targets = np.arange(1,num_targets)
    elif anchor_idx == num_targets -1:
        targets = np.arange(0,num_targets-1)
    else:
        targets = np.concatenate([np.arange(0,anchor_idx),np.arange(anchor_idx+1,num_targets)])
    diff_target_idx = np.random.randint(low = 0,high = num_targets-1)
    diff_target = targets[diff_target_idx]
    
    class_idx_start_end = labels_start_end_train_legit[diff_target,:]
    lo = min(class_idx_start_end[0],class_idx_start_end[1]+1)
    hi = max(class_idx_start_end[0],class_idx_start_end[1]+1)
    idx_from_diff_target = np.random.randint(low = lo,high = hi)
    img = X_train_legit[idx_from_diff_target,:]
    
    return img,diff_target

# Sample batches

In [10]:
# Don't sample negative image from the same category as the positive image (e.g. google and google drive)
# Create clusters of same-company websites (e.g. all microsoft websites)


targets_file = open(data_path+'targets.txt', "r")
all_targets = targets_file.read()
all_targets = all_targets.splitlines()

def get_idx_of_target(target_name,all_targets):
    for i in range(0,len(all_targets)):
        if all_targets[i] == target_name:
            found_idx = i
            return found_idx
        
#targets names of parent and sub websites
target_lists = [['microsoft','ms_outlook','ms_office','ms_bing','ms_onedrive','ms_skype'],['apple','itunes','icloud'],['google','google_drive'],['alibaba','aliexpress']]

def get_associated_targets_idx(target_lists,all_targets):
    sub_target_lists_idx = []
    parents_ids = []
    for i in range(0,len(target_lists)):
        target_list = target_lists[i]
        parent_target = target_list[0]
        one_target_list = []
        parent_idx = get_idx_of_target(parent_target,all_targets)
        parents_ids.append(parent_idx)
        for child_target in target_list[1:]:
            child_idx = get_idx_of_target(child_target,all_targets)
            one_target_list.append(child_idx)
        sub_target_lists_idx.append(one_target_list)
    return parents_ids,sub_target_lists_idx 

parents_ids,sub_target_lists_idx  = get_associated_targets_idx(target_lists,all_targets)

def check_if_same_category(img_label1,img_label2):
    if_same = 0
    if img_label1 in parents_ids:
        if img_label2 in sub_target_lists_idx[parents_ids.index(img_label1)]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[0]:
        if img_label2 in sub_target_lists_idx[0] or img_label2 == parents_ids[0]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[1]:
        if img_label2 in sub_target_lists_idx[1] or img_label2 == parents_ids[1]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[2]:
        if img_label2 in sub_target_lists_idx[2] or img_label2 == parents_ids[2]:
            if_same = 1
    return if_same


In [11]:
#sample triplets
def get_batch(batch_size,num_targets):
   
    # initialize 3 empty arrays for the input image batch
    h = X_train_legit.shape[1]
    w = X_train_legit.shape[2]
    triple=[np.zeros((batch_size, h, w,3)) for i in range(3)]

    for i in range(0,batch_size):
        img_idx_pair1 = pick_first_img_idx(labels_start_end_train_legit,num_targets)
        triple[0][i,:,:,:] = X_train_legit[img_idx_pair1,:]
        img_label = int(y_train_legit[img_idx_pair1])
        
        # get image for the second: positive
        triple[1][i,:,:,:] = pick_pos_img_idx(0.15,img_label)
            
        # get image for the thrid: negative from legit
        # don't sample from the same cluster
        img_neg,label_neg = pick_neg_img(img_label,num_targets)
        while check_if_same_category(img_label,label_neg) == 1:
            img_neg,label_neg = pick_neg_img(img_label,num_targets)

        triple[2][i,:,:,:] = img_neg
          
    return triple


# Training

In [12]:
def save_keras_model(model):
    model.save(output_dir+saved_model_name+'.h5')
    print("Saved model to disk")

In [13]:
print("Starting training process!")
print("-------------------------------------")
import datetime
targets_train = np.zeros([batch_size,1])
ts1=datetime.datetime.now()
for i in range(1, n_iter):
    inputs=get_batch(batch_size,num_targets)
    loss_value=model.train_on_batch(inputs,targets_train)
    
    ts2=datetime.datetime.now()
    td = ts2-ts1
    print("\n ------------- \n")
    print("Iteration:{}. Loss: {}, total seconds:{}".format(i,loss_value,td.total_seconds()))
    
    if i % save_interval == 0:
        save_keras_model(model)
        
    if i % lr_interval ==0:
        start_lr = 0.99*start_lr
        K.set_value(model.optimizer.lr, start_lr)

save_keras_model(model)

Starting training process!
-------------------------------------

 ------------- 

Iteration:1. Loss: 2.4008398056030273, total seconds:44.718169

 ------------- 

Iteration:2. Loss: 2.3957672119140625, total seconds:80.267797

 ------------- 

Iteration:3. Loss: 2.352172613143921, total seconds:116.002439

 ------------- 

Iteration:4. Loss: 2.3483076095581055, total seconds:151.621697

 ------------- 

Iteration:5. Loss: 2.341432809829712, total seconds:186.685806

 ------------- 

Iteration:6. Loss: 2.1973721981048584, total seconds:222.186072

 ------------- 

Iteration:7. Loss: 2.1984875202178955, total seconds:257.972663

 ------------- 

Iteration:8. Loss: 2.1891939640045166, total seconds:293.621283

 ------------- 

Iteration:9. Loss: 1.6334692239761353, total seconds:329.344997

 ------------- 

Iteration:10. Loss: 1.4664392471313477, total seconds:364.867348

 ------------- 

Iteration:11. Loss: 1.181606411933899, total seconds:400.134624

 ------------- 

Iteration:12. Loss


 ------------- 

Iteration:99. Loss: 1.246909260749817, total seconds:3528.354583

 ------------- 

Iteration:100. Loss: 1.3498928546905518, total seconds:3564.824464

 ------------- 

Iteration:101. Loss: 1.5236797332763672, total seconds:3600.249185

 ------------- 

Iteration:102. Loss: 0.9839403629302979, total seconds:3636.224399

 ------------- 

Iteration:103. Loss: 1.3532990217208862, total seconds:3671.431009

 ------------- 

Iteration:104. Loss: 1.3689615726470947, total seconds:3706.488168

 ------------- 

Iteration:105. Loss: 1.562462329864502, total seconds:3742.098487

 ------------- 

Iteration:106. Loss: 1.467820167541504, total seconds:3777.539391

 ------------- 

Iteration:107. Loss: 1.599478006362915, total seconds:3812.99731

 ------------- 

Iteration:108. Loss: 1.521063208580017, total seconds:3848.295143

 ------------- 

Iteration:109. Loss: 1.0900036096572876, total seconds:3883.779072

 ------------- 

Iteration:110. Loss: 1.2241624593734741, total seconds


 ------------- 

Iteration:196. Loss: 1.414196252822876, total seconds:6968.502232

 ------------- 

Iteration:197. Loss: 1.0758498907089233, total seconds:7003.828419

 ------------- 

Iteration:198. Loss: 1.2650930881500244, total seconds:7039.271827

 ------------- 

Iteration:199. Loss: 1.1402990818023682, total seconds:7074.876739

 ------------- 

Iteration:200. Loss: 1.210273265838623, total seconds:7109.693132

 ------------- 

Iteration:201. Loss: 1.233960509300232, total seconds:7144.890817

 ------------- 

Iteration:202. Loss: 1.6921489238739014, total seconds:7180.660721

 ------------- 

Iteration:203. Loss: 1.3752808570861816, total seconds:7215.741981

 ------------- 

Iteration:204. Loss: 1.1739625930786133, total seconds:7250.788872

 ------------- 

Iteration:205. Loss: 0.8811624050140381, total seconds:7286.246639

 ------------- 

Iteration:206. Loss: 1.1774035692214966, total seconds:7321.350491

 ------------- 

Iteration:207. Loss: 0.9453326463699341, total sec


 ------------- 

Iteration:293. Loss: 0.6333701014518738, total seconds:10389.685149

 ------------- 

Iteration:294. Loss: 0.38499128818511963, total seconds:10424.651712

 ------------- 

Iteration:295. Loss: 0.4224739074707031, total seconds:10459.760739

 ------------- 

Iteration:296. Loss: 0.16384361684322357, total seconds:10494.784893

 ------------- 

Iteration:297. Loss: 0.16374967992305756, total seconds:10530.110284

 ------------- 

Iteration:298. Loss: 0.19134032726287842, total seconds:10564.970351

 ------------- 

Iteration:299. Loss: 2.188098430633545, total seconds:10599.803475

 ------------- 

Iteration:300. Loss: 0.495937317609787, total seconds:10635.98496

 ------------- 

Iteration:301. Loss: 1.650710105895996, total seconds:10672.05024

 ------------- 

Iteration:302. Loss: 1.7355337142944336, total seconds:10707.250591

 ------------- 

Iteration:303. Loss: 1.6044446229934692, total seconds:10742.19537

 ------------- 

Iteration:304. Loss: 1.491577625274658


 ------------- 

Iteration:389. Loss: 0.2919360399246216, total seconds:13772.074219

 ------------- 

Iteration:390. Loss: 0.2616523504257202, total seconds:13806.864925

 ------------- 

Iteration:391. Loss: 0.164906844496727, total seconds:13842.003061

 ------------- 

Iteration:392. Loss: 0.3620915412902832, total seconds:13877.150637

 ------------- 

Iteration:393. Loss: 1.2258875370025635, total seconds:13912.329058

 ------------- 

Iteration:394. Loss: 0.44175073504447937, total seconds:13947.572351

 ------------- 

Iteration:395. Loss: 0.20415765047073364, total seconds:13983.287268

 ------------- 

Iteration:396. Loss: 0.19899293780326843, total seconds:14018.368301

 ------------- 

Iteration:397. Loss: 0.3807406425476074, total seconds:14054.247109

 ------------- 

Iteration:398. Loss: 0.3623165786266327, total seconds:14090.349731

 ------------- 

Iteration:399. Loss: 0.7355426549911499, total seconds:14125.734537

 ------------- 

Iteration:400. Loss: 0.15990599989

...


 ------------- 

Iteration:579. Loss: 0.14611274003982544, total seconds:20464.654307

 ------------- 

Iteration:580. Loss: 0.21453973650932312, total seconds:20499.852174

 ------------- 

Iteration:581. Loss: 0.351398766040802, total seconds:20534.857904

 ------------- 

Iteration:582. Loss: 0.14568376541137695, total seconds:20569.764391

 ------------- 

Iteration:583. Loss: 0.513900637626648, total seconds:20604.730188

 ------------- 

Iteration:584. Loss: 0.14557690918445587, total seconds:20639.600085

 ------------- 

Iteration:585. Loss: 0.21696409583091736, total seconds:20675.040034

 ------------- 

Iteration:586. Loss: 0.40397343039512634, total seconds:20711.183945

 ------------- 

Iteration:587. Loss: 0.2363758385181427, total seconds:20746.649348

 ------------- 

Iteration:588. Loss: 0.15137028694152832, total seconds:20781.433088

 ------------- 

Iteration:589. Loss: 0.145310640335083, total seconds:20816.38591

 ------------- 

Iteration:590. Loss: 0.1452577412


 ------------- 

Iteration:674. Loss: 0.4276472330093384, total seconds:23799.927343

 ------------- 

Iteration:675. Loss: 0.1717613935470581, total seconds:23835.148621

 ------------- 

Iteration:676. Loss: 0.14089082181453705, total seconds:23870.565507

 ------------- 

Iteration:677. Loss: 0.546359121799469, total seconds:23905.591831

 ------------- 

Iteration:678. Loss: 0.14079268276691437, total seconds:23940.688648

 ------------- 

Iteration:679. Loss: 0.21989203989505768, total seconds:23976.009367

 ------------- 

Iteration:680. Loss: 0.1406942903995514, total seconds:24011.046391

 ------------- 

Iteration:681. Loss: 0.1406453251838684, total seconds:24046.060001

 ------------- 

Iteration:682. Loss: 0.19852784276008606, total seconds:24081.244752

 ------------- 

Iteration:683. Loss: 0.4529917240142822, total seconds:24116.257329

 ------------- 

Iteration:684. Loss: 0.140498086810112, total seconds:24151.459063

 ------------- 

Iteration:685. Loss: 0.23122860491


 ------------- 

Iteration:769. Loss: 0.13646014034748077, total seconds:27140.238654

 ------------- 

Iteration:770. Loss: 0.3595331907272339, total seconds:27175.41649

 ------------- 

Iteration:771. Loss: 0.39684250950813293, total seconds:27211.228492

 ------------- 

Iteration:772. Loss: 0.4070461690425873, total seconds:27252.443413

 ------------- 

Iteration:773. Loss: 0.1362759917974472, total seconds:27289.165408

 ------------- 

Iteration:774. Loss: 0.31422024965286255, total seconds:27328.797911

 ------------- 

Iteration:775. Loss: 0.13700006902217865, total seconds:27366.841131

 ------------- 

Iteration:776. Loss: 0.15761995315551758, total seconds:27407.650187

 ------------- 

Iteration:777. Loss: 0.4322875440120697, total seconds:27446.249109

 ------------- 

Iteration:778. Loss: 0.26993316411972046, total seconds:27484.686705

 ------------- 

Iteration:779. Loss: 0.19002199172973633, total seconds:27524.587597

 ------------- 

Iteration:780. Loss: 0.1450495


 ------------- 

Iteration:864. Loss: 0.34261149168014526, total seconds:30613.449715

 ------------- 

Iteration:865. Loss: 0.13218434154987335, total seconds:30650.78956

 ------------- 

Iteration:866. Loss: 0.1441420018672943, total seconds:30686.694869

 ------------- 

Iteration:867. Loss: 0.1830146312713623, total seconds:30723.436308

 ------------- 

Iteration:868. Loss: 0.15958797931671143, total seconds:30760.651383

 ------------- 

Iteration:869. Loss: 0.16056643426418304, total seconds:30796.822161

 ------------- 

Iteration:870. Loss: 0.1319694221019745, total seconds:30837.486175

 ------------- 

Iteration:871. Loss: 0.3218633830547333, total seconds:30874.80756

 ------------- 

Iteration:872. Loss: 0.13188447058200836, total seconds:30913.121234

 ------------- 

Iteration:873. Loss: 0.20059192180633545, total seconds:30950.347211

 ------------- 

Iteration:874. Loss: 0.1317993551492691, total seconds:30989.200141

 ------------- 

Iteration:875. Loss: 0.200506806


 ------------- 

Iteration:959. Loss: 0.12829695641994476, total seconds:34053.767872

 ------------- 

Iteration:960. Loss: 0.128257155418396, total seconds:34088.649595

 ------------- 

Iteration:961. Loss: 0.12821733951568604, total seconds:34123.664814

 ------------- 

Iteration:962. Loss: 0.12817750871181488, total seconds:34158.424459

 ------------- 

Iteration:963. Loss: 0.3724704384803772, total seconds:34193.439739

 ------------- 

Iteration:964. Loss: 0.12809759378433228, total seconds:34228.557102

 ------------- 

Iteration:965. Loss: 0.12805750966072083, total seconds:34263.459247

 ------------- 

Iteration:966. Loss: 0.12801747024059296, total seconds:34298.558574

 ------------- 

Iteration:967. Loss: 0.21442894637584686, total seconds:34333.397051

 ------------- 

Iteration:968. Loss: 0.12793749570846558, total seconds:34368.199659

 ------------- 

Iteration:969. Loss: 0.12789754569530487, total seconds:34403.147959

 ------------- 

Iteration:970. Loss: 0.12785

# Calculate Embeddings

In [14]:
shared_model = model.layers[3]

whitelist_emb = shared_model.predict(X_train_legit,batch_size=64)
np.save(output_dir+'whitelist_emb',whitelist_emb)
np.save(output_dir+'whitelist_labels',y_train_legit )

phishing_emb = shared_model.predict(all_imgs_test,batch_size=64)
np.save(output_dir+'phishing_emb',phishing_emb)
np.save(output_dir+'phishing_labels',all_labels_test )
